In [34]:
!pip install azureml.sdk
import azureml.core
from azureml.core import Workspace, Datastore, Experiment
import azureml.data
from azureml.data.azure_storage_datastore import AzureFileDatastore, AzureBlobDatastore
from azureml.data.data_reference import DataReference
from azureml.train.dnn import TensorFlow
from azureml.core.compute import ComputeTarget, AmlCompute
from azureml.core.compute_target import ComputeTargetException
from azureml.widgets import RunDetails
import os

  Using cached https://files.pythonhosted.org/packages/b4/1e/c212b6f5b7fc4e8dbc80032c9bb0e0b0653dfb7171fb56d94b4442fb6877/azureml_sdk-1.0.23-py3-none-any.whl


You are using pip version 18.0, however version 19.0.3 is available.
You should consider upgrading via the 'pip install --upgrade pip' command.


ModuleNotFoundError: No module named 'azureml.widgets'

In [3]:
ws = Workspace.from_config()
print('Workspace name: ' + ws.name, 
      'Azure region: ' + ws.location, 
      'Subscription id: ' + ws.subscription_id, 
      'Resource group: ' + ws.resource_group, sep='\n')

Note, we have launched a browser for you to login. For old experience with device code, use "az login --use-device-code"


Found the config file in: /Users/Amrut/config.json
Performing interactive authentication. Please follow the instructions on the terminal.


You have logged in. Now let us find all the subscriptions to which you have access...


Interactive authentication successfully completed.
Workspace name: AccidentDetection
Azure region: eastus
Subscription id: e298a653-a33f-4b32-9ec2-2adfbd3b649d
Resource group: CCGroup7Resources


In [35]:
ds = Datastore.get(ws, datastore_name='dashcamstreetimages')

In [36]:
ds.as_mount()

$AZUREML_DATAREFERENCE_dashcamstreetimages

In [37]:
cluster_name = "crash"

try:
    compute_target = ComputeTarget(workspace=ws, name=cluster_name)
    print('Found existing compute target')
except ComputeTargetException:
    print('Creating a new compute target...')
    compute_config = AmlCompute.provisioning_configuration(vm_size='STANDARD_NC6', 
                                                           max_nodes=4)

    # create the cluster
    compute_target = ComputeTarget.create(ws, cluster_name, compute_config)

    # can poll for a minimum number of nodes and for a specific timeout. 
    # if no min node count is provided it uses the scale settings for the cluster
    compute_target.wait_for_completion(show_output=True, min_node_count=None, timeout_in_minutes=20)

# use get_status() to get a detailed status for the current cluster. 
print(compute_target.get_status().serialize())

compute_targets = ws.compute_targets
for name, ct in compute_targets.items():
    print(name, ct.type, ct.provisioning_state)


Found existing compute target
{'currentNodeCount': 0, 'targetNodeCount': 0, 'nodeStateCounts': {'preparingNodeCount': 0, 'runningNodeCount': 0, 'idleNodeCount': 0, 'unusableNodeCount': 0, 'leavingNodeCount': 0, 'preemptedNodeCount': 0}, 'allocationState': 'Steady', 'allocationStateTransitionTime': '2019-04-18T23:12:02.954000+00:00', 'errors': None, 'creationTime': '2019-04-18T23:11:51.292045+00:00', 'modifiedTime': '2019-04-18T23:12:07.921232+00:00', 'provisioningState': 'Succeeded', 'provisioningStateTransitionTime': None, 'scaleSettings': {'minNodeCount': 0, 'maxNodeCount': 4, 'nodeIdleTimeBeforeScaleDown': 'PT120S'}, 'vmPriority': 'Dedicated', 'vmSize': 'STANDARD_NC6'}
GPU AmlCompute Succeeded
crash AmlCompute Succeeded


In [38]:
# Train using Tensorflow
from azureml.train.dnn import TensorFlow
exp = Experiment(workspace=ws, name='AccidentDetection')
script_params = {
    'train_path': ws.get_default_datastore().as_mount(),
    '--mode': 'train',
    '--gpu': 0,
}

tf_est = TensorFlow(source_directory='/Users/Amrut/Desktop/CarCrashDetector/',
                    script_params=script_params,
                    compute_target=compute_target,
                    entry_script='accident.py',
                    conda_packages=['cv2', 'sys', 'matplotlib','numpy','os', 'time','pdb','argparse'], # in case you need scikit-learn in train.py
                    use_gpu=True)
run = exp.submit(tf_est)

framework_version is not specified, defaulting to version 1.12.
Submitting /Users/Amrut/Desktop/CarCrashDetector directory for run. The size of the directory >= 25 MB, so it can take a few minutes.


In [40]:
run.get_details() 

os.makedirs('./CarCrashModel', exist_ok=True)

for f in run.get_file_names():
    if f.startswith('outputs/CarCrashModel'):
        output_file_path = os.path.join('./CarCrashModel', f.split('/')[-1])
        print('Downloading from {} to {} ...'.format(f, output_file_path))
        run.download_file(name=f, output_file_path=output_file_path)                    